# MIMIC-EYE 1.0 - An Eye-Tracking Data Analytics Toolkit for REFLACX and EYE GAZE

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import numpy as np
import os
import sys
import shutil
import pickle as pkl
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from mimic.Patient import Patient
from mimic.Xray import Xray
from MIMIC_EYE import MIMIC_EYE 
from Constants import Constants as c
from data.groundtruth.BoundingBox import BoundingBox
from data.groundtruth.Ellipse import Ellipse

from util.mimic_data_processing import *

In [3]:
# starting program
mimic_eye = MIMIC_EYE()

In [4]:
gaze_file = pd.read_csv( "/Volumes/SD_DISK/eye_gaze.csv")
reflacx_metadata = pd.read_csv("/Volumes/SD_DISK/XAMI-MIMICv2.0/spreadsheets/REFLACX/metadata.csv")

In [5]:
c.CACHE = {
						"REFLACX" : 0,							# counts how many patients are in REFLACX dataset
						"EYE_GAZE" : 0,							# counts how many patients are in EYE GAZE dataset
						"OVERLAP" : [],							# lists the patient ids that overlap both datasets
						"PATIENTS" : {},						# dictionary containing all information about a single patient
						"XRAY_TO_PATIENT" 	: {},		# dictionary mapping an XRayID to a PatientID
						"XRAY_TO_DIAGNOSIS" : {},		# dictionary mapping an XRayID to a Diagnosis (Eye Gaze only)
						"XRAY_TO_ABNORMALITY" : {},	# dictionary mapping an XRayID to an Abnormality (REFLACX only)
	}

In [6]:
PATIENTS_DIC = {}
XRAY_TO_PATIENT = {}
XRAY_TO_DIAGNOSIS = {}
XRAY_TO_FIXATIONS = {}
XRAY_TO_GAZE = {}
XRAY_TO_ABNORMALITY = {}

In [7]:
# check which column ("Normal", "CHF" or "Pneumonia") has a 1 in the eye_gaze_df dataframe
def get_diagnosis(row):
    diagnosis = "Normal"
    if row.CHF == 1:
        diagnosis = "CHF"
    if row.pneumonia == 1:
        diagnosis = "Pneumonia"
    return diagnosis

def getLabelsFromREFLACX(df, abnormalities):
  
  abnormalities_lst = []
  for abn in range(0,len(abnormalities)):
    if str(df[abn]) == "True":
      abnormalities_lst.append(abnormalities[abn])
  return abnormalities_lst


In [8]:
eye_gaze_sheet = os.path.join(c.DATASET_PATH, "spreadsheets", 'EyeGaze', "master_sheet_with_updated_stayId.csv")
eye_gaze_df = pd.read_csv( eye_gaze_sheet )

# extract all patient keys from EYE GAZE dataset
patient_keys = eye_gaze_df['patient_id'].values.tolist()
patient_keys = list(set(patient_keys))
for patient_key in patient_keys:

    c.CACHE["EYE_GAZE"] = c.CACHE["EYE_GAZE"] + 1

    patient_dirs = os.listdir(c.PATIENT_PATH(c.DATASET_PATH, patient_key))

    if patient_key == 12629893:
      print("patient 12629893")
      print(patient_dirs)
      
    if "REFLACX" in patient_dirs:
      c.CACHE["REFLACX"] = c.CACHE["REFLACX"] + 1
      c.CACHE["OVERLAP"].append(patient_key)

    # performs an inner join between the eye_gaze dataframe and the MIMIC core data on the patient_id key
    # this allows us to obtain all information about a patient
    # if there are multiple records of the patient (due to several visits to the hospital), we record the 
    # patient's information of his last visit to the hospital
    mimic_core = join_patients_mimic_core_data( patient_key, eye_gaze_df)

    # create a new Patient
    patient = Patient(mimic_core["patient_id"].tail(1).values[0], \
                      mimic_core["gender_x"].tail(1).values[0], \
                      mimic_core["language"].tail(1).values[0], \
                      mimic_core["marital_status"].tail(1).values[0],\
                      mimic_core["race"].tail(1).values[0], \
                      mimic_core["anchor_age_x"].tail(1).values[0], \
                      mimic_core["anchor_year"].tail(1).values[0], \
                      mimic_core["dod"].tail(1).values[0], \
                      mimic_core["anchor_year_group"].tail(1).values[0], {}, {}  )

    # get all CXR images associated to patient with patient_key
    xray_lst = []
    patient_df = eye_gaze_df[ eye_gaze_df[ "patient_id"] == patient_key ]
    for row in patient_df.itertuples():
      xray_id = row.dicom_id
      study_id = "s"+str(row.study_id)
      report = read_CXR_report( patient_key, study_id )
      diagnosis = get_diagnosis(row)
    
      XRAY_TO_DIAGNOSIS[xray_id] = diagnosis

      # a patient can have many imagaeology studies, and each study is composed of at least one XRay image
      # one xray is given by an image file, a textual report with all the findings and diagnosis
      # and a set of 
      xray_dicom_path = c.MIMIC_DICOM_PATH(c.DATASET_PATH, patient_key, study_id, xray_id)
    
      fixations_dict = {}
      gaze_dict = {}
      ell_annotation_dict = {}
      bbox_lst = []
      bboxes_df = pd.read_csv( c.EYE_GAZE_BBOX_PATH(c.DATASET_PATH, patient_key) )
      for bbox in bboxes_df.itertuples():
        bbox = BoundingBox(bbox.x1, bbox.x2, bbox.y1, bbox.y2, bbox.bbox_name)
        bbox_lst.append( bbox )

      cxr = Xray(xray_id, study_id, report, diagnosis, xray_dicom_path)
      cxr.setBboxList(bbox_lst)
      
      fixations_dict[study_id + "_" + xray_id] = c.EYE_GAZE_FIXATIONS_PATH(c.DATASET_PATH, patient_key)
      gaze_dict[study_id + "_" + xray_id] = c.EYE_GAZE_RAW_PATH(c.DATASET_PATH, patient_key)

      if "REFLACX" in patient_dirs:

        metadata = pd.read_csv(c.REFLACX_METADATA(c.DATASET_PATH, patient_key))
        ref_row = metadata[ metadata["dicom_id"] == xray_id ]

        if patient_key == 12629893:
          print(metadata)
          print(ref_row)
   
        ell_annotation_dict = cxr.getAbnormalityDict()
        for ref in ref_row.itertuples():
          refl_id = ref_row["id"].values[0]
          
          temp = []
          if refl_id in ell_annotation_dict.keys():
            temp = ell_annotation_dict[refl_id]
          ellipse_annotations = pd.read_csv(c.REFLACX_ELLIPSE_PATH(c.DATASET_PATH, patient_key, refl_id))
          for ell in ellipse_annotations.itertuples():
            ell_labels = getLabelsFromREFLACX(ell, ellipse_annotations.columns.to_list())
            for ell_label in ell_labels:
              temp.append(Ellipse(ell.xmin, ell.xmax, ell.ymin, ell.ymax, ell_label, ell.certainty ))
          
          ell_annotation_dict[refl_id] = temp
          fixations_dict[refl_id + "_" + xray_id] = c.REFLACX_FIXATIONS_PATH(c.DATASET_PATH, patient_key, refl_id)
          gaze_dict[refl_id + "_" + xray_id] = c.REFLACX_RAW_PATH(c.DATASET_PATH, patient_key, refl_id)
        
          cxr.setAbnormalityDict(ell_annotation_dict)


          # moving gaze files to the XAMI-MIMIC folder
          # source_gaze_file_path = c.REFLACX_GAZE_SOURCE_PATH(c.REFLACX_SOURCE, row.id)
          # destinatination_gaze_folder_path = c.REFLACX_RAW_PATH(c.DATASET_PATH, row.subject_id, row.id)
          # destinatination_gaze_folder_path.replace("gaze.csv", "")
          
          # print(f"Copying file from {source_gaze_file_path} to {destinatination_gaze_folder_path}")
          # shutil.copy(source_gaze_file_path, destinatination_gaze_folder_path )
        
      cxr.setFixationsDict(fixations_dict)
      cxr.setGazeDict(gaze_dict)
      

      xray_lst.append( cxr )

      # get all fixations associated to the current xray
      # print(mimic_core["patient_id"].tail(1).values[0])
      # print(xray_id)
      # gaze_df = gaze_file[ gaze_file["DICOM_ID"] == xray_id ]
      # gaze_df.to_csv( cxr.getEyegazeRawPathList()[0], index=False )
      
      # print("saving EYE GAZE gaze file to " +  cxr.getEyegazeRawPathList()[0])


      XRAY_TO_PATIENT[xray_id] = patient

    # check if a patient key exists in PATIENTS_DIC
    if patient_key in PATIENTS_DIC:
      print(f"Patient {patient_key} already exists in PATIENTS_DIC. Processing DICOM ID {xray_id}")
      print(f"Current DICOM IDs")
      print(PATIENTS_DIC[patient_key].get_xray_ids())
      print("---------------------------------------------------------------------\n")
    
    patient.setXray_lst(xray_lst)
    PATIENTS_DIC[patient_key] = patient
    
c.CACHE["PATIENTS"] = PATIENTS_DIC
c.CACHE["XRAY_TO_PATIENT"] = XRAY_TO_PATIENT
c.CACHE["XRAY_TO_DIAGNOSIS"] = XRAY_TO_DIAGNOSIS
c.CACHE["OVERLAP"] = list(set(c.CACHE["OVERLAP"]))



patient 12629893
['EyeGaze', '.DS_Store', '._.DS_Store', 'Core', 'CXR-JPG', 'ED', 'CXR-DICOM', 'REFLACX']


In [10]:
patient_dirs

['EyeGaze', '.DS_Store', '._.DS_Store', 'Core', 'CXR-JPG', 'ED', 'CXR-DICOM']

In [11]:
mimic_eye.save_state()

In [10]:
ref_patient = c.CACHE["PATIENTS"][12629893]
ref_xrays = ref_patient.getXray_lst()
cxr = ref_xrays[0]
cxr.getInfo()

{'ID': 'f0256c5c-a68a55ba-51e7cc78-c6aa82eb-7d0eb5fe',
 'study_id': 's51632471',
 'report': 'FINAL REPORT\nCHEST RADIOGRAPH PERFORMED ON ___\n\nCOMPARISON:  ___.\n\nCLINICAL HISTORY:  Shortness of breath and chest pain.\n\nFINDINGS:  PA and lateral views of the chest were obtained.  Midline\nsternotomy wires and mediastinal clips are again noted. There is a background\nof emphysema better appreciated on the prior radiographs. There has been\ninterval development of pulmonary vascular congestion and mild pulmonary\nedema.  Trace bilateral pleural effusions are present.  No pneumothorax.\nCardiomediastinal silhouette is stable.  Bony structures are intact.\n\nIMPRESSION:  Emphysema with superimposed pulmonary edema.  Trace bilateral\npleural effusions.\n',
 'diagnosis': 'CHF',
 'dicom_path': '/Volumes/SD_DISK/XAMI-MIMICv2.0/patient_12629893/CXR-DICOM/s51632471/f0256c5c-a68a55ba-51e7cc78-c6aa82eb-7d0eb5fe.dcm',
 'jpg_path': '/Volumes/SD_DISK/XAMI-MIMICv2.0/patient_12629893/CXR-JPG/s516324

In [9]:
c.CACHE

{'REFLACX': 45,
 'EYE_GAZE': 1038,
 'OVERLAP': [12629893,
  11052935,
  13593993,
  17532555,
  19157903,
  14492434,
  15634195,
  14150037,
  18087960,
  12658584,
  14794396,
  15216540,
  16449190,
  15225006,
  18659631,
  19346228,
  13483060,
  15284020,
  18734137,
  18426683,
  11329595,
  11004477,
  12651710,
  12917695,
  15857729,
  11297219,
  10402372,
  17927113,
  10160202,
  14987339,
  17037515,
  11537996,
  19584206,
  17023312,
  12530259,
  19565653,
  16831446,
  13473495,
  15752803,
  19856485,
  17167982,
  18673777,
  11892979,
  13977589,
  19273599],
 'PATIENTS': {17440770: <mimic.Patient.Patient at 0x120eec850>,
  13953026: <mimic.Patient.Patient at 0x120d7b160>,
  14211078: <mimic.Patient.Patient at 0x108c6e370>,
  13139976: <mimic.Patient.Patient at 0x120f000d0>,
  13023241: <mimic.Patient.Patient at 0x120f001f0>,
  11390987: <mimic.Patient.Patient at 0x120f00190>,
  19152908: <mimic.Patient.Patient at 0x120f08d00>,
  14415891: <mimic.Patient.Patient at

In [24]:
xray_id = "116c787f-3423c257-151f5943-ae7bd2ff-b7f0c496"

In [13]:
p = "/Volumes/SD_DISK/XAMI-MIMICv2.0/patient_18087960/REFLACX/P300R801796/anomaly_location_ellipses.csv"
df = pd.read_csv(p)
row = df.iloc[0,:]

In [8]:
reflacx_data = reflacx_metadata[ reflacx_metadata["dicom_id"] == "fa771fa1-d9571d07-bff8f655-327734a7-6e10b29d" ]
reflacx_data

,id,split,eye_tracking_data_discarded,image,dicom_id,subject_id,image_size_x,image_size_y,Airway wall thickening,Atelectasis,Consolidation,Emphysema,Enlarged cardiac silhouette,Fibrosis,Fracture,Groundglass opacity,Mass,Nodule,Other,Pleural effusion,Pleural thickening,Pneumothorax,Pulmonary edema,Quality issue,Support devices,Wide mediastinum,Abnormal mediastinal contour,Acute fracture,Enlarged hilum,Hiatal hernia,High lung volume / emphysema,Interstitial lung disease,Lung nodule or mass,Pleural abnormality
545,P300R081391,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p10/...,fa771fa1-d9571d07-bff8f655-327734a7-6e10b29d,10002428,2544,3056,NaN,4,4,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,True,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


------------------
------------------


In [56]:
ellipse_annotations

,xmin,ymin,xmax,ymax,certainty,Abnormal mediastinal contour,Acute fracture,Atelectasis,Consolidation,Enlarged cardiac silhouette,Enlarged hilum,Groundglass opacity,Hiatal hernia,High lung volume / emphysema,Interstitial lung disease,Lung nodule or mass,Other,Pleural abnormality,Pneumothorax,Pulmonary edema,Support devices
0,893,1142,1351,1487,5.0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
1,417,1321,1196,1958,5.0,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False


In [58]:
ell_annotation_lst

In [57]:
for ellipse in ell_annotation_lst:
  print(ellipse.getCoordinates())
  print(ellipse.getLabel())


[893, 1351, 1142, 1487]
Consolidation
[417, 1196, 1321, 1958]
Consolidation
[417, 1196, 1321, 1958]
Enlarged cardiac silhouette


In [7]:


reflacx_fixation_lst = []
reflacx_gaze_lst = []
ell_annotation_lst = []
for row in reflacx_data.itertuples():
  # adding the path to the fixations and gaze data from reflacx dataset to the XRay object
  reflacx_fixation_lst.append(c.REFLACX_FIXATIONS_PATH(c.DATASET_PATH, row.subject_id, row.id))
  reflacx_gaze_lst.append(c.REFLACX_RAW_PATH(c.DATASET_PATH, row.subject_id, row.id))
  
  ellipse_annotations = pd.read_csv(c.REFLACX_ELLIPSE_PATH(c.DATASET_PATH, row.subject_id, row.id))
  for ell in ellipse_annotations.itertuples():
    ell_labels = getLabelsFromREFLACX(ell)
    for ell_label in ell_labels:
      ell_annotation_lst.append(Ellipse(ell.xmin, ell.xmax, ell.ymin, ell.ymax, ell_label, ell.certainty ))
  

AttributeError: 'Pandas' object has no attribute 'columns'

In [29]:
reflacx_fixation_lst

[]

In [21]:
df.columns

Index(['xmin', 'ymin', 'xmax', 'ymax', 'certainty',
       'Abnormal mediastinal contour', 'Acute fracture', 'Atelectasis',
       'Consolidation', 'Enlarged cardiac silhouette', 'Enlarged hilum',
       'Groundglass opacity', 'Hiatal hernia', 'High lung volume / emphysema',
       'Interstitial lung disease', 'Lung nodule or mass', 'Other',
       'Pleural abnormality', 'Pneumothorax', 'Pulmonary edema',
       'Support devices'],
      dtype='object')

In [1]:
# save state
mimic_eye.save_state()


NameError: name 'mimic_eye' is not defined